In [44]:

import json
import pprint

import requests


# Testing HL Api for PUSH transfer

---

## Consts

In [45]:
data_space_provider = "http://127.0.0.1:1234"
data_space_consumer = "http://127.0.0.1:1235"
context_broker = "http://fiware-orion:1026"
streaming_processor = "http://streaming-testing-service:1237"

catalog_id = ""
dataservice_id = ""
agreement_id = ""
agreement_pid = ""

## Setup Catalog, Dataservice and agreements

In [46]:
payload = {
    "foaf:homepage": "My catalog in Dataspace provider",
    "dct:title": "My catalog in Dataspace provider"
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
catalog_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': 'c96317c7-34c2-48bf-8e18-cb91c284719e',
 '@type': 'dcat:Catalog',
 'dcat:dataset': [],
 'dcat:keyword': '',
 'dcat:service': [],
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': 'c96317c7-34c2-48bf-8e18-cb91c284719e',
 'dct:issued': '2024-11-26T14:12:58.997708',
 'dct:modified': None,
 'dct:title': 'My catalog in Dataspace provider',
 'dspace:extraFields': None,
 'dspace:participantId': None,
 'foaf:homepage': 'My catalog in Dataspace provider',
 'odrl:hasPolicy': None}


In [47]:
suscription_payload = {
    "description": "Air Quality Unit Subscription",
    "subject": {
        "entities":[
            {"idPattern":".*","type":"AirQualityUnit"}],
        "condition": {
            "attrs":["CO"]
        }
    },
    "notification": {
        "http": {
            "url":"$data_url",
        },
    },
    "expires":"2040-01-01T14:00:00.00Z",
    "throttling":5
}
payload = {
    "dcat:endpointURL": context_broker + "/v2/subscriptions",
    "dcat:endpointDescription": json.dumps(suscription_payload),
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/catalogs/" + catalog_id + "/data-services"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
dataservice_id = response_as_json["@id"]

pprint.pprint(response.json())

{'@context': 'https://w3id.org/dspace/2024/1/context.json',
 '@id': '654da3c7-9508-4768-b64d-f9823ce179a8',
 '@type': 'dcat:DataService',
 'dcat:endpointDescription': '{"description": "Air Quality Unit Subscription", '
                             '"subject": {"entities": [{"idPattern": ".*", '
                             '"type": "AirQualityUnit"}], "condition": '
                             '{"attrs": ["CO"]}}, "notification": {"http": '
                             '{"url": "$data_url"}}, "expires": '
                             '"2040-01-01T14:00:00.00Z", "throttling": 5}',
 'dcat:endpointURL': 'http://fiware-orion:1026/v2/subscriptions',
 'dcat:keyword': '',
 'dcat:theme': '',
 'dct:conformsTo': None,
 'dct:creator': None,
 'dct:description': [],
 'dct:identifier': '654da3c7-9508-4768-b64d-f9823ce179a8',
 'dct:issued': '2024-11-26T14:12:59.027257',
 'dct:modified': None,
 'dct:title': None,
 'dspace:extraFields': None,
 'odrl:hasPolicy': None}


In [48]:
payload = {
    "dataServiceId": dataservice_id
}
headers = {
    "Content-Type": "application/json",
}
url = data_space_provider + "/api/v1/agreements"
response = requests.request("POST", url, headers=headers, data=json.dumps(payload))
response_as_json = response.json()
agreement_id = response_as_json["agreement_id"]
agreement_pid = "urn:uuid:" + agreement_id

pprint.pprint(response.json())

{'agreement_id': '985c9168-b50a-40b1-a7d5-47086d3cb97b',
 'data_service_id': '654da3c7-9508-4768-b64d-f9823ce179a8',
 'identity': None,
 'identity_token': None}


## Setup transfer

In [49]:
url = data_space_consumer + "/api/v1/setup-transfer"

payload = ""
headers = {}
response = requests.request("POST", url, headers=headers, data=payload)
callbackAddress = response.json()["callbackAddress"]
callbackId = response.json()["callbackId"]
consumerPid = response.json()["consumerPid"]

pprint.pprint(response.json())

{'callbackAddress': 'http://ds-consumer:1235/3d328f05-173f-42a0-bdaf-6a2d70656430',
 'callbackId': '3d328f05-173f-42a0-bdaf-6a2d70656430',
 'consumerPid': 'urn:uuid:f2d4267a-0afb-4442-a920-b66d9c89f090'}


## Request transfer

In [50]:
url = data_space_consumer + "/api/v1/request-transfer"

payload = json.dumps({
    "agreementId": agreement_pid,
    "format": "fiware+push",
    "dataAddress": {
        "@type": "dspace:DataAddress",
        "dspace:endpoint": "http://host.docker.internal:5000/data",
        "dspace:endpointType": "HTTP",
        "dspace:endpointProperties": []
    },
    "callbackAddress": callbackAddress,
    "callbackId": callbackId,
    "consumerPid": consumerPid
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

pprint.pprint(response.json())


{'consumerPid': 'urn:uuid:f2d4267a-0afb-4442-a920-b66d9c89f090',
 'transferProcess': {'@context': 'https://w3id.org/dspace/2024/1/context.json',
                     '@type': 'dspace:TransferProcess',
                     'dspace:consumerPid': 'urn:uuid:f2d4267a-0afb-4442-a920-b66d9c89f090',
                     'dspace:providerPid': 'urn:uuid:b744e172-3e1a-4093-ba03-958a5e42acc3',
                     'dspace:state': 'dspace:REQUESTED'}}


#### For seeing how the push service is working, please check the logs in the streaming-testing-service

## Server

In [59]:
from flask import Flask, request, jsonify
import json
import time

app = Flask(__name__)

@app.route('/data', methods=['POST'])
def receive_data():
    data = json.loads(request.get_data())
    timestamp = int(time.time() * 1000)
    filename = f"data/fiware_stream_{timestamp}.json"
    with open(filename, 'w') as f:
        json.dump(data, f)

    return jsonify({"status": "Data received"}), 200

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.153.52.22:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Nov/2024 15:20:28] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2024 15:20:28] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2024 15:20:33] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2024 15:20:33] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2024 15:20:38] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2024 15:20:38] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2024 15:20:43] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2024 15:20:43] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2024 15:20:48] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2024 15:20:48] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2024 15:20:54] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2024 15:20:54] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2024 15:20:59] "POST /data HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov